In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

from dotenv import load_dotenv
import os

load_dotenv()
    
# 공공데이터포털에서 발급받은 API 키
api_key = os.environ.get('api_key')
# 요청할 API URL
url = f'https://apis.data.go.kr/1160100/service/GetStockSecuritiesInfoService/getStockPriceInfo?serviceKey={api_key}&resultType=xml'

params = {
    "numOfRows": 100,
    "pageNo": 1,
    "beginBasDt": "20240401",
    "beginMrktTotAmt": 1000000000000
}

# API 요청 보내기
response = requests.get(url, params=params)

# 응답 상태 코드 확인
if response.status_code == 200:
    # 응답 데이터 파싱 (XML 형식)
    root = ET.fromstring(response.content)
    
    # totalCount와 numOfRows 요소 찾기
    total_count_element = root.find('.//totalCount')
    num_of_rows_element = root.find('.//numOfRows')

    if total_count_element is not None and num_of_rows_element is not None:
        total_count = int(total_count_element.text)
        rows = int(num_of_rows_element.text)

        print(total_count)
        print(rows)

        all_items = []

        for page in range(1, (total_count // rows) + 2):  # 페이지 계산 수정
            params['pageNo'] = page
            response = requests.get(url, params=params)

            if response.status_code == 200:
                root = ET.fromstring(response.content)
                items = root.findall('.//item')
                for item in items:
                    basDt_element = item.find('basDt')
                    srtnCd_element = item.find('srtnCd')
                    clpr_element = item.find('clpr')
                    mrktTotAmt_element = item.find('mrktTotAmt')
                    trqu_element = item.find('trqu')
                    
                    if basDt_element is not None and srtnCd_element is not None and clpr_element is not None and trqu_element is not None:
                        basDt = basDt_element.text
                        srtnCd = srtnCd_element.text
                        clpr = clpr_element.text
                        mrktTotAmt = mrktTotAmt_element.text
                        trqu = trqu_element.text
                        all_items.append((basDt, srtnCd, clpr, mrktTotAmt, trqu))
            else:
                print(f'페이지 {page} 요청 실패: {response.status_code}')
                break

        # DataFrame 생성
        df = pd.DataFrame(all_items, columns=['date', 'code', 'price', 'amount', 'volume'])

        # 엑셀 파일로 저장
        # df.to_excel('종목코드_종목명_종목상세.xlsx', index=False)
        print('엑셀 파일 저장 완료: 종목코드_종목명_종목상세.xlsx')

    else:
        print('totalCount 또는 numOfRows 요소를 찾을 수 없습니다.')

else:
    print(f'API 요청 실패: {response.status_code}')

13413
100


KeyboardInterrupt: 

In [17]:
int_array = df[df['code']=='000080']['price'].astype(int)
print(int_array.mean())
print(int_array.std())

20556.875
454.57090475173237


In [28]:
df

,date,code,price,amount,volume
0,20240612,000080,20750,1455272428250,148519
1,20240612,000100,74300,5959533455200,724899
2,20240612,000120,97400,2221922305600,132849
3,20240612,000150,223000,3684815205000,222874
4,20240612,000210,59200,1240588332800,87303
...,...,...,...,...,...
12848,20240401,403870,51500,4271108416000,1883641
12849,20240401,417200,26150,1769117032850,9701712
12850,20240401,450080,131500,9071556167000,368245
12851,20240401,454910,84900,5503216302000,938867


In [31]:
df[df['code']=='000080']['price'].astype(int)

0        20750
280      20650
558      20600
836      20100
1110     19990
1382     20100
1652     20100
1923     20100
2193     19940
2464     19990
2737     19950
3013     20100
3286     19980
3557     20150
3829     20100
4103     19980
4373     20100
4645     20200
4916     20200
5187     20300
5461     20400
5732     20400
6000     20250
6270     20250
6540     20250
6811     21100
7077     21050
7345     21050
7611     21050
7878     20900
8145     21000
8406     21000
8669     21100
8928     21050
9188     20650
9447     21000
9705     20800
9962     20900
10220    21300
10479    21100
10740    21550
11001    21450
11262    20350
11526    20600
11792    20450
12059    20900
12323    20650
12589    20800
Name: price, dtype: int64